In [1]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import cv2

In [2]:
def logistic_map(x, r):
    return r * x * (1 - x)

def generate_key(seed, r, length):
    key = np.zeros(length)
    key[0] = seed
    for i in range(1, length):
        key[i] = logistic_map(key[i-1], r)
    return key

In [3]:
def encrypt(img, key, block_size=8):
    # Pad the image to make sure its dimensions are divisible by the block size
    height, width, channels = img.shape
    padding_height = (block_size - height % block_size) % block_size
    padding_width = (block_size - width % block_size) % block_size
    padded_img = cv2.copyMakeBorder(img, 0, padding_height, 0, padding_width, cv2.BORDER_CONSTANT, value=0)
    
    # Reshape the padded image into blocks of size block_size x block_size
    padded_height, padded_width, channels = padded_img.shape
    num_blocks_height = padded_height // block_size
    num_blocks_width = padded_width // block_size
    blocks = np.zeros((num_blocks_height, num_blocks_width, block_size, block_size, channels), dtype=np.uint8)
    for i in range(num_blocks_height):
        for j in range(num_blocks_width):
            blocks[i, j] = padded_img[i*block_size:(i+1)*block_size, j*block_size:(j+1)*block_size]
    
    # Encrypt each block using the key
    encrypted_blocks = np.zeros_like(blocks)
    key_index = 0
    for i in range(num_blocks_height):
        for j in range(num_blocks_width):
            for c in range(channels):
                for k in range(block_size):
                    for l in range(block_size):
                        encrypted_blocks[i, j, k, l, c] = blocks[i, j, k, l, c] ^ int(key[key_index]*255)
                        key_index = (key_index + 1) % len(key)
    
    # Reshape the encrypted blocks back into an image
    encrypted_img = np.zeros_like(padded_img)
    for i in range(num_blocks_height):
        for j in range(num_blocks_width):
            encrypted_img[i*block_size:(i+1)*block_size, j*block_size:(j+1)*block_size] = encrypted_blocks[i, j]
    
    # Crop the padded area to get the original image size
    encrypted_img = encrypted_img[:height, :width, :]
    
    return encrypted_img

In [4]:
def decrypt(encrypted_img, key, block_size=8):
    # Pad the encrypted image to make sure its dimensions are divisible by the block size
    height, width, channels = encrypted_img.shape
    padding_height = (block_size - height % block_size) % block_size
    padding_width = (block_size - width % block_size) % block_size
    padded_encrypted_img = cv2.copyMakeBorder(encrypted_img, 0, padding_height, 0, padding_width, cv2.BORDER_CONSTANT, value=0)
    
    # Reshape the padded encrypted image into blocks of size block_size x block_size
    padded_height, padded_width, channels = padded_encrypted_img.shape
    num_blocks_height = padded_height // block_size
    num_blocks_width = padded_width // block_size
    blocks = np.zeros((num_blocks_height, num_blocks_width, block_size, block_size, channels), dtype=np.uint8)
    for i in range(num_blocks_height):
        for j in range(num_blocks_width):
            blocks[i, j] = padded_encrypted_img[i*block_size:(i+1)*block_size, j*block_size:(j+1)*block_size]
    
    # Decrypt each block using the key
    decrypted_blocks = np.zeros_like(blocks)
    key_index = 0
    for i in range(num_blocks_height):
        for j in range(num_blocks_width):
            for c in range(channels):
                for k in range(block_size):
                    for l in range(block_size):
                        decrypted_blocks[i, j, k, l, c] = blocks[i, j, k, l, c] ^ int(key[key_index]*255)
                        key_index = (key_index + 1) % len(key)
    
    # Reshape the decrypted blocks back into an image
    decrypted_img = np.zeros_like(padded_encrypted_img)
    for i in range(num_blocks_height):
        for j in range(num_blocks_width):
            decrypted_img[i*block_size:(i+1)*block_size, j*block_size:(j+1)*block_size] = decrypted_blocks[i, j]
    
    # Crop the padded area to get the original image size
    decrypted_img = decrypted_img[:height, :width, :]
    
    return decrypted_img

In [6]:
# Example usage
seed = 0.5
r = 3.9
length = 1024
key = generate_key(seed, r, length)

# Load image
img = plt.imread('test.jpg')

# Encrypt image
encrypted_img = encrypt(img, key)

# Save encrypted image
plt.imsave('encrypted.jpg', encrypted_img)

In [7]:
# Decrypt image
decrypted_img = decrypt(encrypted_img, key)

# Save decrypted image
plt.imsave('decrypted.jpg', decrypted_img)